In [13]:
#train_data 正確率
#帶入論文model 試試


import torch
import numpy as np
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


#利用pandas 讀取csv檔
import pandas as pd
df=pd.read_csv('~/Desktop/train.csv') #2010筆 (train:1340 test:670)
#df=pd.read_csv('~/Desktop/bias_paper_train_mle.csv')  #5615筆 train:3715 test:1900
#df=pd.read_csv('~/Desktop/bias contest/data/bias_paper_train_delete.csv')  #5613筆 train:3714 test:1899
#df=pd.read_csv('~/Desktop/outgroup_paper_train_mle.csv') #5307筆 train:3537 test:1770
#df=pd.read_csv('~/Desktop/outgroup_paper_train_delete.csv') #5305筆 train:3535 test:1770

data=df.values

import random
# 設置隨機種子
np.random.seed(25)

#將資料打亂
np.random.shuffle(data)

'''
#bias_paper_train.csv
X=data[:,[2,3,4,5,6,7]]
y=data[:,1]
'''

#53 self_invest
X=data[:,[49,50,51,52,54,56,58,61,62,63,64]]
#X=data[:,[49,53,54,56,58,61,62,63,64]]
y=data[:,46]
#y=data[:,47]


train_num=1340
test_num=670

X_train=X[:train_num, :]
X_test=X[train_num:,:]
y_train=y[:train_num]
y_test=y[train_num:]

'''
X_train=X[:1810, :]
X_test=X[1810:,:]
y_train=y[:1810]
y_test=y[1810:]
'''
#X_train裡的元素是object 無法轉成tensor所以將其先轉為float
X_train=X_train.astype(float)
X_test=X_test.astype(float)
y_train=y_train.astype(float)
y_test=y_test.astype(float)

X_train=torch.Tensor(X_train)
X_test=torch.Tensor(X_test)
y_train=torch.Tensor(y_train)
y_test=torch.Tensor(y_test)

#建立資料與標籤一組的dataset
train_data=TensorDataset(X_train, y_train)
test_data=TensorDataset(X_test, y_test)

#指定資料的批次大小與建立Dataloader
size=1340
load_train=DataLoader(train_data, batch_size=size, shuffle=True)
load_test=DataLoader(test_data, batch_size=size, shuffle=False) #test不用打亂資料

In [14]:
# 設置隨機種子
torch.manual_seed(52)

#model_1 linear
'''
model=nn.Sequential()
model.add_module('fc1',nn.Linear(12,1))
'''

#model_5 polynimial
feature=len(X_train[0])
model=nn.Sequential()
model.add_module('fc1',nn.Linear(feature,60))
model.add_module('relu1',nn.ReLU())
model.add_module('fc5',nn.Linear(60,50))
model.add_module('relu5',nn.ReLU())
model.add_module('fc2',nn.Linear(50,30))
model.add_module('relu2',nn.ReLU())
model.add_module('fc3',nn.Linear(30,20))
model.add_module('relu3',nn.ReLU())
Linear=nn.Linear(20,1, bias=True)
model.add_module('fc4', Linear)


from torch import optim
from sklearn.metrics import r2_score
#建立loss function
#loss_fn=nn.L1Loss() #輸出需要long 所以我們在上面的tag加上 .type(torch.LongTensor)
loss_fn=nn.MSELoss()

#建立優化函數
optimizer=optim.Adam(model.parameters(), lr=0.005,weight_decay=0.1)
#optimizer=optim.SGD(model.parameters(), lr=0.01)

#開始訓練

def train(epoch):
    model.train() #將模型轉成訓練模式

    for data, targets in load_train:
        optimizer.zero_grad()
        outputs=model(data)
        outputs=outputs.reshape(size,)
        loss=loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

    print ("epoch {}: 結束\n".format(epoch))
    
    model.eval()
    correct=0
    '''
    with torch.no_grad(): #不需計算微分
        for data, targets in load_train:
            outputs=model(data)
            outputs=outputs.reshape(1,)
            for i in range(len(outputs)):
                if abs((outputs[i]-targets[i]).item())<0.05:
                    correct+=1
        data_num=len(load_train.dataset)
        print("\n正確率:{}/{} ({:.0f}%)\n".format(correct,data_num,100.*correct/data_num))
    '''
    # 預測值
    y_train_pred = model(X_train)

    # 將預測值轉換為numpy陣列
    y_train_pred = y_train_pred.detach().numpy()

    # 將實際目標值轉換為numpy陣列
    y_true = y_train.numpy()

    # 計算R^2 score
    r2 = r2_score(y_true, y_train_pred)
    print("Train R^2 score:", r2)

    # 預測值
    y_test_pred = model(X_test)

    # 將預測值轉換為numpy陣列
    y_test_pred = y_test_pred.detach().numpy()

    # 將實際目標值轉換為numpy陣列
    y_true = y_test.numpy()

    # 計算R^2 score
    r2 = r2_score(y_true, y_test_pred)
    print("Test R^2 score:", r2)
    adjust_r2=1-(1-r2)*((len(X_test)-1)/(len(X_test)-len(X_test[0])-1))
    print("Adjusted R^2 score:", adjust_r2)
    return r2, adjust_r2


def test():
    model.eval()
    correct=0

    with torch.no_grad(): #不需計算微分
        for data, targets in load_test:
            outputs=model(data)
            outputs=outputs.reshape(size,)
            '''
            print(type(outputs))
            print(type(targets))
            print(outputs.shape)
            print(targets.shape)
            '''
            for i in range(len(outputs)):
                if abs((outputs[i]-targets[i]).item())<0.1:
                    correct+=1
    data_num=len(load_test.dataset)
    print("\n正確率:{}/{} ({:.0f}%)\n".format(correct,data_num,100.*correct/data_num))


In [15]:
for epoch in range(1000):
    train(epoch)

epoch 0: 結束

Train R^2 score: 0.0006702107767679877
Test R^2 score: 0.0008944602834214388
Adjusted R^2 score: -0.015807911961080556
epoch 1: 結束

Train R^2 score: 0.008476865499520092
Test R^2 score: 0.008606169631722338
Adjusted R^2 score: -0.007967283459540786
epoch 2: 結束

Train R^2 score: 0.01512891288773588
Test R^2 score: 0.015038978527874614
Adjusted R^2 score: -0.0014269352049420814
epoch 3: 結束

Train R^2 score: 0.02142830647267835
Test R^2 score: 0.021178707863896085
Adjusted R^2 score: 0.004815433983201278
epoch 4: 結束

Train R^2 score: 0.028010197895679267
Test R^2 score: 0.027457002466205194
Adjusted R^2 score: 0.01119868487825415
epoch 5: 結束

Train R^2 score: 0.035264162944648736
Test R^2 score: 0.03434639752076174
Adjusted R^2 score: 0.018203252190561736
epoch 6: 結束

Train R^2 score: 0.04350964823431824
Test R^2 score: 0.042235587313629974
Adjusted R^2 score: 0.026224328134982433
epoch 7: 結束

Train R^2 score: 0.0532235395440247
Test R^2 score: 0.051533152193363296
Adjusted R

In [16]:
'''
import random

R_2=0
Adjusted_R2=0
for i in range(10): 
    print("This is data {} ".format(i+1))
    df=pd.read_csv('~/Desktop/train.csv')
    data=df.values

    #將資料打亂
    random.shuffle(data)
    print(data.shape)
    X=data[:,[49,50,51,52,53,54,56,58,61,62,63,64]]
    y=data[:,46]
        
    #將資料切成十份
    X_train=np.delete(X, slice(201*i,201*(i+1)), axis=0)
    X_test=X[201*i:201*(i+1)]
    y_train=np.delete(y, slice(201*i,201*(i+1)), axis=0)
    y_test=y[201*i:201*(i+1)]

    #X_train裡的元素是object 無法轉成tensor所以將其先轉為float
    X_train=X_train.astype(float)
    X_test=X_test.astype(float)
    y_train=y_train.astype(float)
    y_test=y_test.astype(float)

    X_train=torch.Tensor(X_train)
    X_test=torch.Tensor(X_test)
    y_train=torch.Tensor(y_train)
    y_test=torch.Tensor(y_test)

    #建立資料與標籤一組的dataset
    train_data=TensorDataset(X_train, y_train)
    test_data=TensorDataset(X_test, y_test)

    #指定資料的批次大小與建立Dataloader
    size=3
    load_train=DataLoader(train_data, batch_size=size, shuffle=True)
    load_test=DataLoader(test_data, batch_size=size, shuffle=False) #test不用打亂資料

    #model_1 linear
    model=nn.Sequential()
    model.add_module('fc1',nn.Linear(12,1))

    from torch import optim
    from sklearn.metrics import r2_score
    #建立loss function
    #loss_fn=nn.L1Loss() #輸出需要long 所以我們在上面的tag加上 .type(torch.LongTensor)
    loss_fn=nn.MSELoss()

    #建立優化函數
    #optimizer=optim.Adam(model.parameters(), lr=0.005)
    optimizer=optim.SGD(model.parameters(), lr=0.01,weight_decay=0.1)
    test()
    for epoch in range(5):
        r2, adjust_r2=train(epoch)
        if epoch==4:
            R_2+=r2
            Adjusted_R2+=adjust_r2
    test() 
print(R_2/10)
print(Adjusted_R2/10)
'''

'\nimport random\n\nR_2=0\nAdjusted_R2=0\nfor i in range(10): \n    print("This is data {} ".format(i+1))\n    df=pd.read_csv(\'~/Desktop/train.csv\')\n    data=df.values\n\n    #將資料打亂\n    random.shuffle(data)\n    print(data.shape)\n    X=data[:,[49,50,51,52,53,54,56,58,61,62,63,64]]\n    y=data[:,46]\n        \n    #將資料切成十份\n    X_train=np.delete(X, slice(201*i,201*(i+1)), axis=0)\n    X_test=X[201*i:201*(i+1)]\n    y_train=np.delete(y, slice(201*i,201*(i+1)), axis=0)\n    y_test=y[201*i:201*(i+1)]\n\n    #X_train裡的元素是object 無法轉成tensor所以將其先轉為float\n    X_train=X_train.astype(float)\n    X_test=X_test.astype(float)\n    y_train=y_train.astype(float)\n    y_test=y_test.astype(float)\n\n    X_train=torch.Tensor(X_train)\n    X_test=torch.Tensor(X_test)\n    y_train=torch.Tensor(y_train)\n    y_test=torch.Tensor(y_test)\n\n    #建立資料與標籤一組的dataset\n    train_data=TensorDataset(X_train, y_train)\n    test_data=TensorDataset(X_test, y_test)\n\n    #指定資料的批次大小與建立Dataloader\n    size=3\n    lo

In [17]:
# 預測值
y_train_pred = model(X_train)

# 將預測值轉換為numpy陣列
y_train_pred = y_train_pred.detach().numpy()

# 將實際目標值轉換為numpy陣列
y_true = y_train.numpy()

# 計算R^2 score
r2 = r2_score(y_true, y_train_pred)
print("Train R^2 score:", r2)

# 預測值
y_test_pred = model(X_test)

# 將預測值轉換為numpy陣列
y_test_pred = y_test_pred.detach().numpy()

# 將實際目標值轉換為numpy陣列
y_true = y_test.numpy()

# 計算R^2 score
r2 = r2_score(y_true, y_test_pred)
print("Test R^2 score:", r2)

Train R^2 score: 0.6150697597285645
Test R^2 score: 0.57757984395537


In [18]:
from sklearn.metrics import mean_squared_error

# 计算RMSE
y_pred_train = model(X_train).detach().numpy()
y_train_numpy=y_train.detach().numpy()
rmse = mean_squared_error(y_train_numpy, y_pred_train, squared=False)
print(f"RMSE: {rmse}")

# 计算RMSE
y_pred_test= model(X_test).detach().numpy()
y_test_numpy=y_test.detach().numpy()
rmse = mean_squared_error(y_test_numpy, y_pred_test, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.8554710149765015
RMSE: 0.8933898210525513
